## 1. 다국어 RAG 시스템 구축 실습

- 실제 RAG 시스템은 다양한 언어의 문서를 다루거나, 여러 언어로 질문을 받는 상황에 놓일 수 있음.
- 다국어 환경에서 RAG 시스템을 구축하는 방법에 대한 예시 확인

**다국어 RAG의 주요 과제:**
- **언어 불일치**: 질문과 문서의 언어가 다를 경우 검색 성능이 저하될 수 있음.
- **임베딩 모델 선택**: 사용하는 임베딩 모델이 대상 언어들을 얼마나 잘 지원하는지가 중요하며, 교차 언어(cross-lingual) 성능이 좋은 모델이 필요할 수 있음.
- **번역 품질 및 비용**: 자동 번역기를 사용할 경우 번역 품질, 지연 시간, 비용 등을 고려해야 함.

### 1-1 언어 교차(cross-lingual) 검색

- 교차 언어 임베딩 모델은 서로 다른 언어의 텍스트라도 의미가 유사하면 벡터 공간에서 가깝게 위치하도록 학습됨. 
- 이러한 모델을 사용하면 한국어로 질문해도 영어 문서를 검색하거나, 그 반대의 경우도 가능하게 됨.

**전략:**
1.  한국어 문서와 영어 문서를 모두 준비.
2.  교차 언어 성능이 우수한 임베딩 모델 (예: OpenAI `text-embedding-3-small`, HuggingFace `BAAI/bge-m3`, Ollama `bge-m3`)을 선택.
3.  모든 문서를 선택한 임베딩 모델로 임베딩하여 하나의 벡터 저장소에 저장.
4.  사용자 질문(한국어 또는 영어)을 동일한 임베딩 모델로 임베딩하여 벡터 저장소에서 유사 문서를 검색.

**장점:**
- 구현이 비교적 간단합니다. 단일 임베딩 모델과 단일 벡터 저장소만 관리하면 됨.

**단점:**
- 임베딩 모델의 교차 언어 성능에 크게 의존하며, 모델 성능이 부족하면 검색 정확도가 떨어질 수 있음
- 동일 언어 내 검색(예: 한국어 질문 -> 한국어 문서)보다 성능이 낮을 수 있음.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(1) 다국어 문서 로드 및 전처리` 예시


In [2]:
from glob import glob
import os 
korean_txt_files = glob(os.path.join('./data', '*_KR.txt')) 
english_txt_files = glob(os.path.join('./data', '*_EN.txt'))

print("한국어 텍스트 파일:", korean_txt_files)
print("영어 텍스트 파일:", english_txt_files)

한국어 텍스트 파일: ['./data\\Rivian_KR.txt', './data\\Tesla_KR.txt']
영어 텍스트 파일: ['./data\\Rivian_EN.txt', './data\\Tesla_EN.txt']


In [3]:
from langchain_community.document_loaders import TextLoader

def load_text_files(txt_files):
    data = []
    for text_file in txt_files:
        # TextLoader는 기본적으로 utf-8을 가정하나, 명시하는 것이 좋음
        loader = TextLoader(text_file, encoding='utf-8') 
        data.extend(loader.load())
    return data

korean_data = load_text_files(korean_txt_files)
english_data = load_text_files(english_txt_files)

print(f"한국어 원본 Document 수: {len(korean_data)}")
print(f"영어 원본 Document 수: {len(english_data)}")

if korean_data:
    print("\n한국어 데이터 샘플 (첫 번째 문서 메타데이터):", korean_data[0].metadata)
    print("한국어 데이터 샘플 (첫 번째 문서 내용 일부):", korean_data[0].page_content[:200])
if english_data:
    print("\n영어 데이터 샘플 (첫 번째 문서 메타데이터):", english_data[0].metadata)
    print("영어 데이터 샘플 (첫 번째 문서 내용 일부):", english_data[0].page_content[:200])

한국어 원본 Document 수: 2
영어 원본 Document 수: 2

한국어 데이터 샘플 (첫 번째 문서 메타데이터): {'source': './data\\Rivian_KR.txt'}
한국어 데이터 샘플 (첫 번째 문서 내용 일부): 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다.

리비안의 

영어 데이터 샘플 (첫 번째 문서 메타데이터): {'source': './data\\Rivian_EN.txt'}
영어 데이터 샘플 (첫 번째 문서 내용 일부): Founded in 2009 by MIT PhD graduate RJ Scaringe, Rivian is an innovative American electric vehicle manufacturer. Initially focused on autonomous electric cars from 2011, Rivian's significant growth ph


In [4]:
from langchain_text_splitters import CharacterTextSplitter
# 문장을 구분하여 분할 (마침표, 느낌표, 물음표 다음에 공백이 오는 경우 문장의 끝으로 판단)
# 이 모델은 교차 언어 성능이 뛰어나므로, 다국어 문서 처리에 적합
# why?? "대규모 다국어 데이터셋으로 사전 학습" 및 "정교한 토크나이저"
text_splitter_multilingual = CharacterTextSplitter.from_tiktoken_encoder(
    model_name="text-embedding-3-small", # 이 모델의 토크나이저 사용
    separator=r"[.!?]\s+",      # 문장 끝 구두점 뒤 공백 기준 분할 (정규식)
    chunk_size=200,              # 목표 청크 토큰 수 (CharacterTextSplitter는 이 값을 엄격히 따르지 않을 수 있음)
    chunk_overlap=20,            # 청크 간 중복 토큰 수
    is_separator_regex=True,     # separator를 정규식으로 해석
    keep_separator=False,        # 구분자(공백) 제거
)

korean_docs_split = []
english_docs_split = []

if korean_data:
    korean_docs_split = text_splitter_multilingual.split_documents(korean_data)
if english_data:
    english_docs_split = text_splitter_multilingual.split_documents(english_data)

print(f"분할된 한국어 문서(청크) 수: {len(korean_docs_split)}")
print(f"분할된 영어 문서(청크) 수: {len(english_docs_split)}")

분할된 한국어 문서(청크) 수: 6
분할된 영어 문서(청크) 수: 4


In [5]:
# 한국어 분할 청크 확인 (처음 2개)
if korean_docs_split:
    print("--- 한국어 분할 청크 샘플 ---")
    for i, doc in enumerate(korean_docs_split[:2]):
        print(f"\n[청크 {i+1}] (길이: {len(doc.page_content)}자, 메타데이터: {doc.metadata})")
        print(doc.page_content[:150])
        if len(doc.page_content) > 150: print("[...]")
else:
    print("분할된 한국어 문서가 없습니다.")

--- 한국어 분할 청크 샘플 ---

[청크 1] (길이: 204자, 메타데이터: {'source': './data\\Rivian_KR.txt'})
2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다[.!?]\s+2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립
[...]

[청크 2] (길이: 178자, 메타데이터: {'source': './data\\Rivian_KR.txt'})
리비안의 초기 프로젝트는 피터 스티븐스가 디자인한 2+2 시트 배열의 미드십 엔진 하이브리드 쿠페 스포츠카 R1(원래 이름은 아베라(Avera))이었습니다[.!?]\s+이 차량은 모듈식 캡슐 구조와 쉽게 교체 가능한 차체 패널을 특징으로 하며, 2013년 말에서 201
[...]


In [6]:
# 영어 분할 청크 확인 (처음 2개)
if english_docs_split:
    print("\n--- 영어 분할 청크 샘플 ---")
    for i, doc in enumerate(english_docs_split[:2]):
        print(f"\n[청크 {i+1}] (길이: {len(doc.page_content)}자, 메타데이터: {doc.metadata})")
        print(doc.page_content[:150])
        if len(doc.page_content) > 150: print("[...]")
else:
    print("분할된 영어 문서가 없습니다.")


--- 영어 분할 청크 샘플 ---

[청크 1] (길이: 757자, 메타데이터: {'source': './data\\Rivian_EN.txt'})
Founded in 2009 by MIT PhD graduate RJ Scaringe, Rivian is an innovative American electric vehicle manufacturer[.!?]\s+Initially focused on autonomous
[...]

[청크 2] (길이: 372자, 메타데이터: {'source': './data\\Rivian_EN.txt'})
Rivian also considered various versions, including a diesel hybrid, a racing version named R1 GT for a Brazilian one-make series, a four-door sedan, a
[...]


`(2) 문서 임베딩 및 벡터저장소에 저장 ` 예시
1) 선택한 교차 언어 임베딩 모델들(OpenAI, HuggingFace, Ollama)을 사용하여 한국어와 영어 문서를 함께 임베딩하고, 각 모델별로 ChromaDB 벡터 저장소에 저장
2) 이렇게 하면 각 임베딩 모델의 교차 언어 검색 성능을 비교 가능능

- `collection_name`: 벡터 저장소 내에서 특정 문서 그룹을 식별하는 이름이며, 모델별로 다른 이름을 사용 가능.
- `persist_directory`: 벡터 저장소 데이터를 디스크에 저장할 경로이며, 지정하면 저장소가 유지되어 재사용 가능..

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_ollama import OllamaEmbeddings


# OpenAI 임베딩 모델 (교차 언어 지원 우수)
embeddings_openai_small_cl = OpenAIEmbeddings(model="text-embedding-3-small")

# Hugging Face 임베딩 모델 (BAAI/bge-m3, 교차 언어 지원 우수)
embeddings_huggingface_bge_m3_cl = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3", 
    model_kwargs={'device': 'cpu'}, # CPU 또는 'cuda' 
    encode_kwargs={'normalize_embeddings': True}
)

# Ollama 임베딩 모델 (bge-m3 또는 nomic-embed-text, 교차 언어 지원 가능성 확인 필요)
# Ollama 서버 및 해당 모델이 준비되어 있어야 함
embeddings_ollama_bge_m3_cl = None
ollama_ready_cl = False
try:
    embeddings_ollama_bge_m3_cl = OllamaEmbeddings(model="bge-m3") 
    # embeddings_ollama_nomic_cl = OllamaEmbeddings(model="nomic-embed-text")
    print("Ollama 임베딩 모델 (bge-m3) 준비 완료.")
    ollama_ready_cl = True
except Exception as e:
    print(f"Ollama 연결 또는 모델 로드 실패 (교차언어용): {e}")
    print("Ollama (bge-m3) 교차언어 예제를 실행하려면 Ollama 서버를 실행하고 'bge-m3' 모델을 pull 해주세요.")

all_split_docs = korean_docs_split + english_docs_split
print(f"총 분할된 문서(청크) 수: {len(all_split_docs)}")

Ollama 임베딩 모델 (bge-m3) 준비 완료.
총 분할된 문서(청크) 수: 10


In [8]:
# 다국어 벡터 저장소 구축
from langchain_chroma import Chroma

db_openai_cl = None
db_huggingface_cl = None
db_ollama_cl = None

if all_split_docs: # 분할된 문서가 있을 경우에만 실행
    print("OpenAI 임베딩으로 벡터 저장소 구축 중...")
    db_openai_cl = Chroma.from_documents(
        documents=all_split_docs, 
        embedding=embeddings_openai_small_cl,
        collection_name="db_openai_crosslingual_v2", # 컬렉션 이름 변경 또는 기존 삭제 후 생성
        persist_directory="./chroma_db_cl", # 디렉토리 구분
    )
    print(f"OpenAI 벡터 저장소 문서 수: {db_openai_cl._collection.count()}")

    print("\nHugging Face (BAAI/bge-m3) 임베딩으로 벡터 저장소 구축 중...")
    db_huggingface_cl = Chroma.from_documents(
        documents=all_split_docs, 
        embedding=embeddings_huggingface_bge_m3_cl,
        collection_name="db_huggingface_crosslingual_v2",
        persist_directory="./chroma_db_cl",
    )
    print(f"Hugging Face 벡터 저장소 문서 수: {db_huggingface_cl._collection.count()}")

    if ollama_ready_cl and embeddings_ollama_bge_m3_cl:
        print("\nOllama (bge-m3) 임베딩으로 벡터 저장소 구축 중...")
        db_ollama_cl = Chroma.from_documents(
            documents=all_split_docs, 
            embedding=embeddings_ollama_bge_m3_cl,
            collection_name="db_ollama_crosslingual_v2",
            persist_directory="./chroma_db_cl",
        )
        print(f"Ollama 벡터 저장소 문서 수: {db_ollama_cl._collection.count()}")
    else:
        print("\nOllama가 준비되지 않아 Ollama 벡터 저장소 구축을 건너뜁니다.")
else:
    print("분할된 문서가 없어 벡터 저장소 구축을 건너뜁니다.")

OpenAI 임베딩으로 벡터 저장소 구축 중...
OpenAI 벡터 저장소 문서 수: 10

Hugging Face (BAAI/bge-m3) 임베딩으로 벡터 저장소 구축 중...
Hugging Face 벡터 저장소 문서 수: 10

Ollama (bge-m3) 임베딩으로 벡터 저장소 구축 중...
Ollama 벡터 저장소 문서 수: 10


`(3) RAG 성능 비교 `  

- 간단한 RAG 체인을 구성하여 각 임베딩 모델 기반의 벡터 저장소가 한국어 질문과 영어 질문에 대해 어떻게 응답하는지 비교

**RAG 체인 구성 요소:**
- **Retriever**: 벡터 저장소에서 유사 문서를 검색하며, `as_retriever()`로 변환하고, `search_kwargs={'k': 2}`로 상위 2개 문서를 가져오도록 설정.
- **Prompt Template**: LLM에 전달할 프롬프트를 정의하며, 컨텍스트(검색된 문서)와 질문을 포함.
- **LLM**: 질문과 컨텍스트를 바탕으로 최종 답변을 생성할 언어 모델 (예: `ChatOpenAI`)
- **Output Parser**: LLM의 출력(주로 `AIMessage` 객체)에서 실제 텍스트 답변만 추출(`StrOutputParser`)
- **RunnablePassthrough / format_docs**: LangChain Expression Language (LCEL)에서 데이터 흐름을 관리하고, 검색된 `Document` 객체 리스트를 LLM 프롬프트에 적합한 문자열 형태로 변환.

In [9]:
# RAG 체인 생성
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based ONLY on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt_template_cl = ChatPromptTemplate.from_template(template)

# 문서 포맷터 함수: Document 객체 리스트를 단일 문자열로 결합
def format_docs_cl(docs):
    return "\n\n".join([d.page_content for d in docs])

# LLM 모델 생성 (답변 생성용)
llm_cl = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 체인 생성 함수 (벡터 저장소를 인자로 받음)
def create_rag_chain_cl(vectorstore):
    if not vectorstore:
        # 벡터 저장소가 None이면, 실행 불가능한 더미 체인을 반환하거나 예외 처리
        # 여기서는 간단히 None을 반환하여 호출하는 쪽에서 확인하도록 함
        return None
        
    retriever = vectorstore.as_retriever(search_kwargs={'k': 2}) # 상위 2개 문서 검색

    rag_chain = (
        {"context": retriever | format_docs_cl , "question": RunnablePassthrough()} # 검색 및 포맷팅
        | prompt_template_cl  # 프롬프트 적용
        | llm_cl              # LLM으로 답변 생성
        | StrOutputParser()   # 출력 파싱 (텍스트만 추출)
    )
    return rag_chain

In [10]:
# 각 벡터 저장소에 대한 RAG 체인 생성
rag_chain_openai_cl = create_rag_chain_cl(db_openai_cl)
rag_chain_huggingface_cl = create_rag_chain_cl(db_huggingface_cl)
rag_chain_ollama_cl = create_rag_chain_cl(db_ollama_cl)

if rag_chain_openai_cl: print("OpenAI RAG 체인 생성 완료")
else: print("OpenAI RAG 체인 생성 실패 (벡터 저장소 없음)")

if rag_chain_huggingface_cl: print("HuggingFace RAG 체인 생성 완료")
else: print("HuggingFace RAG 체인 생성 실패 (벡터 저장소 없음)")

if rag_chain_ollama_cl: print("Ollama RAG 체인 생성 완료")
else: print("Ollama RAG 체인 생성 실패 (벡터 저장소 없음 또는 Ollama 준비 안됨)")

OpenAI RAG 체인 생성 완료
HuggingFace RAG 체인 생성 완료
Ollama RAG 체인 생성 완료


In [11]:
# 한국어 쿼리에 대한 성능 평가
query_ko_cl = "테슬라 창업자는 누구인가요?" # 예시 문서에 관련 내용이 있어야 함
print(f"\n--- 한국어 쿼리: '{query_ko_cl}' ---")

if rag_chain_openai_cl:
    output_openai_ko = rag_chain_openai_cl.invoke(query_ko_cl)
    print(f"OpenAI 응답 (KO): {output_openai_ko}")
else:
    print("OpenAI RAG 체인이 없어 실행 불가")

if rag_chain_huggingface_cl:
    output_huggingface_ko = rag_chain_huggingface_cl.invoke(query_ko_cl)
    print(f"Hugging Face (bge-m3) 응답 (KO): {output_huggingface_ko}")
else:
    print("HuggingFace RAG 체인이 없어 실행 불가")

if rag_chain_ollama_cl:
    output_ollama_ko = rag_chain_ollama_cl.invoke(query_ko_cl)
    print(f"Ollama (bge-m3) 응답 (KO): {output_ollama_ko}")
else:
    print("Ollama RAG 체인이 없어 실행 불가")


--- 한국어 쿼리: '테슬라 창업자는 누구인가요?' ---
OpenAI 응답 (KO): 마틴 에버하드와 마크 타페닝입니다.
Hugging Face (bge-m3) 응답 (KO): 마틴 에버하드와 마크 타페닝입니다.
Ollama (bge-m3) 응답 (KO): 마틴 에버하드와 마크 타페닝입니다.


In [19]:
# 영어 쿼리에 대한 성능 평가
query_en_cl = "Who is the founder of Tesla?" # 예시 문서에 관련 내용이 있어야 함
print(f"\n--- 영어 쿼리: '{query_en_cl}' ---")

if rag_chain_openai_cl:
    output_openai_en = rag_chain_openai_cl.invoke(query_en_cl)
    print(f"OpenAI 응답 (EN): {output_openai_en}")
else:
    print("OpenAI RAG 체인이 없어 실행 불가")

if rag_chain_huggingface_cl:
    output_huggingface_en = rag_chain_huggingface_cl.invoke(query_en_cl)
    print(f"Hugging Face (bge-m3) 응답 (EN): {output_huggingface_en}")
else:
    print("HuggingFace RAG 체인이 없어 실행 불가")

if rag_chain_ollama_cl:
    output_ollama_en = rag_chain_ollama_cl.invoke(query_en_cl)
    print(f"Ollama (bge-m3) 응답 (EN): {output_ollama_en}")
else:
    print("Ollama RAG 체인이 없어 실행 불가")


--- 영어 쿼리: 'Who is the founder of Tesla?' ---
OpenAI 응답 (EN): 마틴 에버하드와 마크 타페닝입니다.
Hugging Face (bge-m3) 응답 (EN): 마틴 에버하드와 마크 타페닝입니다.
Ollama (bge-m3) 응답 (EN): 마틴 에버하드와 마크 타페닝입니다.


### 1-2 언어 감지 및 자동번역 통합 

- 이 전략은 주로 단일 언어(예: 한국어)로 구성된 문서 저장소를 가지고 있을 때, 다양한 언어의 사용자 질문을 처리하기 위해 사용
- 또는 그 반대의 경우도 가능

**전략:**
1.  **언어 감지**: 사용자 질문의 언어를 감지 (예: `langdetect` 라이브러리 사용).
2.  **질문 번역**: 감지된 질문 언어가 문서 저장소의 주 언어와 다르면, 질문을 문서 저장소의 언어로 번역(예: `deepl` API 사용).
3.  **RAG 처리**: 번역된 질문을 사용하여 일반적인 RAG 체인을 통해 답변을 생성 (이때 답변은 문서 저장소의 언어로 생성됨).
4.  **답변 번역**: 생성된 답변의 언어가 원래 질문의 언어와 다르면, 답변을 원래 질문의 언어로 다시 번역

**장점:**
- 단일 언어에 최적화된 임베딩 모델과 LLM을 활용하여 해당 언어에서의 검색 및 답변 생성 품질을 극대화할 수 있음.
- 다양한 언어의 사용자 질문을 지원할 수 있음.

**단점:**
- **번역 품질 의존성**: 번역기의 성능에 따라 전체 시스템의 품질이 크게 좌우됨. 번역 오류는 잘못된 검색 결과나 부정확한 답변으로 이어질 수 있음.
- **지연 시간 증가**: 질문과 답변 번역 과정에서 추가적인 API 호출로 인해 전체 응답 시간이 늘어남.
- **번역 비용**: 상용 번역 API(예: DeepL, Google Translate) 사용 시 비용이 발생.
- **언어 감지 오류**: 언어 감지가 정확하지 않으면 불필요하거나 잘못된 번역이 발생 가능성.

`(1) 한국어 문서 벡터저장소 로드 (또는 생성)`  

- 한국어 문서만으로 구성된 벡터 저장소가 이미 있다고 가정.
- 만약 없다면, 한국어 문서(`korean_docs_split`)와 한국어에 강한 임베딩 모델(예: `OpenAIEmbeddings`, 또는 한국어 특화 HuggingFace 모델)을 사용하여 새로 생성할 수 있음.

- `chroma_test`라는 이름의 기존 컬렉션을 로드하려고 시도.
- 이 컬렉션은 이전 단계나 한국어 데이터로 이미 만들어졌다고 가정.
- 만약 해당 컬렉션이 없다면, `Chroma.from_documents`를 사용하여 새로 만들어야 함.

In [13]:
# 한국어 문서로 저장되어 있는 벡터 저장소 로드 또는 생성
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_for_ko_store = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

# 기존에 'chroma_test' 컬렉션이 한국어 데이터로 만들어져 있다고 가정.
# 없다면, korean_docs_split을 사용해 새로 생성해야 함.
# 예시: db_korean_only = Chroma.from_documents(documents=korean_docs_split, embedding=embeddings_for_ko_store, collection_name="korean_store_v1", persist_directory="./chroma_db_ko_only")

COLLECTION_NAME_KO_ONLY = "db_openai_crosslingual_v2" # (한국어+영어 데이터 포함)
PERSIST_DIR_KO_ONLY = "./chroma_db_cl"

try:
    # 여기서는 1-1에서 만든 db_openai_cl (한국어+영어 문서 포함)을 재사용
    # 순수 한국어 저장소를 원한다면, 해당 저장소를 로드하거나 새로 만들어야 함.
    vectorstore_ko_trans = Chroma(
        embedding_function=embeddings_for_ko_store,
        collection_name=COLLECTION_NAME_KO_ONLY, # 1-1에서 사용한 OpenAI 컬렉션 이름
        persist_directory=PERSIST_DIR_KO_ONLY    # 1-1에서 사용한 디렉토리
    )
    print(f"'{COLLECTION_NAME_KO_ONLY}' 벡터 저장소 로드 완료. 문서 수: {vectorstore_ko_trans._collection.count()}")
    # 이 저장소는 실제로는 한국어와 영어가 섞여있지만, 지금은 한국어 중심 저장소로 간주하고 진행
    # 이상적으로는 순수 한국어 문서로 구성된 저장소를 사용하는 것이 이 시나리오에 더 적합
except Exception as e:
    print(f"벡터 저장소 '{COLLECTION_NAME_KO_ONLY}' 로드 실패: {e}")
    print("이전 단계에서 해당 이름의 컬렉션이 생성되었는지, 경로가 올바른지 확인하세요.")
    vectorstore_ko_trans = None

'db_openai_crosslingual_v2' 벡터 저장소 로드 완료. 문서 수: 10


**언어 감지 및 번역 도구 설정**

- `langdetect`: 텍스트의 언어를 감지
- `deepl`: 고품질 번역을 제공하는 API 서비스

In [14]:
import os
import deepl
from langdetect import detect, DetectorFactory

# 언어 감지 결과의 일관성을 위해 시드 설정 (선택 사항)
DetectorFactory.seed = 0

deepl_api_key = os.getenv('DEEPL_API_KEY')
translator = None
if deepl_api_key:
    translator = deepl.Translator(deepl_api_key)
    print("DeepL 번역기 초기화 완료.")
else:
    print("DEEPL_API_KEY가 설정되지 않았습니다. 번역 기능을 사용하려면 API 키를 설정하세요.")

def detect_and_translate(text: str, target_lang_deepl: str = 'KO', target_lang_detect: str = 'ko'):
    if not translator:
        print("번역기가 설정되지 않아 원본 텍스트를 반환합니다.")
        # 번역기 없어도 언어 감지는 시도
        try:
            detected_lang = detect(text)
            return text, detected_lang
        except Exception as e:
            print(f"언어 감지 실패: {e}. 원본 텍스트 및 'unknown' 반환.")
            return text, "unknown"

    try:
        detected_lang = detect(text) # langdetect는 'ko', 'en' 등 소문자 코드를 반환
    except Exception as e:
        print(f"언어 감지 실패: {e}. 원본 텍스트 사용.")
        return text, "unknown"

    # target_lang_deepl ('EN', 'KO', 'EN-US' 등)을 langdetect 스타일 ('en', 'ko')로 변환하여 비교하기 위함
    simple_target_deepl_code = ''
    temp_target_deepl_upper = target_lang_deepl.upper()
    if temp_target_deepl_upper.startswith('EN'): # 'EN', 'EN-US', 'EN-GB' 등
        simple_target_deepl_code = 'en'
    elif temp_target_deepl_upper == 'KO':
        simple_target_deepl_code = 'ko'
    # 다른 언어에 대한 규칙 추가 가능
    else:
        # 일반적인 경우 (예: 'FR', 'JA') DeepL 코드는 2자리이므로 앞 두 글자를 소문자로 사용
        if len(target_lang_deepl) >= 2:
            simple_target_deepl_code = target_lang_deepl[:2].lower()
        else: # 매우 드문 경우
            simple_target_deepl_code = target_lang_deepl.lower()

    # 감지된 언어(소문자)와 실제 목표 언어(단순화된 소문자 코드)를 비교
    if detected_lang.lower() != simple_target_deepl_code:
        print(f"번역 필요: 감지된 언어 '{detected_lang}' -> 목표 언어 '{target_lang_deepl}'")
        try:
            # DeepL은 'EN-US', 'EN-GB' 등을 지원. 단순히 'EN'으로 하면 기본 'EN-US'로 될 수 있음.
            # langdetect가 'en'을 반환하면, DeepL의 'EN-US'로 매핑하는 것이 안전할 수 있음.
            target_lang_deepl_actual = target_lang_deepl # 기본값 설정
            if target_lang_deepl.upper() == 'EN': # 만약 영어로 번역해야 한다면
                target_lang_deepl_actual = 'EN-US' # 미국 영어로 지정 (DeepL 기본 동작과 일치시키거나 명시적 지정)
            # 다른 target_lang_deepl 값들은 그대로 사용 (예: 'KO', 'FR', 'DE')
            # (위의 if문에서 target_lang_deepl_actual = target_lang_deepl 로 이미 설정됨)

            result = translator.translate_text(text, target_lang=target_lang_deepl_actual)
            return str(result), detected_lang
        except Exception as e:
            print(f"번역 실패: {e}. 원본 텍스트 사용.")
            return text, detected_lang
    else: # 감지된 언어와 (단순화된) 목표 언어가 같으면 번역 안 함
        print(f"번역 불필요: 감지된 언어 '{detected_lang}' (단순화: '{detected_lang.lower()}') == 목표 언어 '{target_lang_deepl}' (단순화: '{simple_target_deepl_code}'). 원본 텍스트 반환.")
        return text, detected_lang

# 문서 번역 테스트
if translator:
    sample_text_en = "Hello. How are you today?"
    translated_text, detected_lang_sample = detect_and_translate(sample_text_en, target_lang_deepl='KO') # target_lang_detect 기본값 'ko' 사용
    print(f"원본 (EN): {sample_text_en}")
    print(f"감지된 언어: {detected_lang_sample}")
    print(f"번역된 텍스트 (KO): {translated_text}")
    print("----")

    sample_text_ko = "안녕하세요. 오늘 기분이 어떠신가요?"
    translated_text_en, detected_lang_sample_ko = detect_and_translate(sample_text_ko, target_lang_deepl='EN') # target_lang_detect 기본값 'ko' 사용
    print(f"원본 (KO): {sample_text_ko}")
    print(f"감지된 언어: {detected_lang_sample_ko}")
    print(f"번역된 텍스트 (EN-US): {translated_text_en}")
else:
    print("DeepL 번역기가 없어 번역 테스트를 건너뜁니다.")

DeepL 번역기 초기화 완료.
번역 필요: 감지된 언어 'en' -> 목표 언어 'KO'
원본 (EN): Hello. How are you today?
감지된 언어: en
번역된 텍스트 (KO): 안녕하세요. 오늘은 어떠세요?
----
번역 필요: 감지된 언어 'ko' -> 목표 언어 'EN'
원본 (KO): 안녕하세요. 오늘 기분이 어떠신가요?
감지된 언어: ko
번역된 텍스트 (EN-US): hello. How are you feeling today?


`(2) RAG 체인 성능 평가 `  

- `detect_and_translate` 함수를 통합하여, 질문과 답변을 필요에 따라 번역하는 RAG 체인을 생성.
- 이 체인은 내부적으로 한국어 컨텍스트를 기반으로 답변을 생성하고, 원래 질문 언어에 맞춰 최종 답변을 번역

In [15]:
lang_rag_chain_translate = None
if vectorstore_ko_trans: # 한국어(중심) 벡터 저장소가 로드되었는지 확인
    retriever_ko_trans = vectorstore_ko_trans.as_retriever(search_kwargs={'k': 2})

    # RAG 체인 구성 (5-1의 프롬프트, LLM, 포맷터 재사용)
    lang_rag_chain_translate = (
        {"context": retriever_ko_trans | format_docs_cl , "question": RunnablePassthrough()}
        | prompt_template_cl # 1-1에서 정의한 프롬프트 템플릿
        | llm_cl             # 1-1에서 정의한 LLM
        | StrOutputParser()
    )
    print("번역 통합 RAG 체인 생성 완료.")
else:
    print("한국어(중심) 벡터 저장소가 없어 번역 통합 RAG 체인을 생성할 수 없습니다.")

# 번역 통합 체인 실행 함수
def run_lang_rag_chain_with_translation(query: str):
    if not lang_rag_chain_translate:
        return "번역 통합 RAG 체인이 준비되지 않았습니다."
    if not translator: # DeepL 번역기가 준비되지 않았으면
        print("DeepL 번역기가 없어 번역 없이 원본 질문으로 RAG를 실행합니다.")
        # 이 경우, 질문과 문서의 언어가 일치하지 않으면 성능이 저하 가능성 
        return lang_rag_chain_translate.invoke(query)

    # 1. 질문 언어 감지 및 한국어(문서 저장소 언어)로 번역
    translated_query, original_lang_code_lc = detect_and_translate(query, target_lang_deepl='KO', target_lang_detect='ko')
    print(f"원본 질문 ({original_lang_code_lc}): {query}")
    if original_lang_code_lc.lower() != 'ko':
        print(f"번역된 질문 (KO): {translated_query}")
    
    # 2. 번역된 (한국어) 질문으로 RAG 체인 실행 -> 답변은 한국어로 생성됨
    output_ko = lang_rag_chain_translate.invoke(translated_query)
    print(f"RAG 답변 (KO): {output_ko}")
    
    # 3. 생성된 한국어 답변을 원래 질문 언어로 다시 번역
    if original_lang_code_lc.lower() != 'ko':
        # DeepL target_lang은 대문자 (예: 'EN-US', 'JA')
        # langdetect는 소문자 (예: 'en', 'ja')
        # DeepL에서 지원하는 형태로 변환 필요
        target_lang_deepl_for_answer = original_lang_code_lc.upper()
        if target_lang_deepl_for_answer == 'EN': # langdetect가 'en'이면
            target_lang_deepl_for_answer = 'EN-US' # DeepL용 'EN-US'로
        # 다른 언어들도 필요시 매핑 규칙 추가 (예: 'ZH-CN' -> 'ZH')

        final_output, _ = detect_and_translate(output_ko, target_lang_deepl=target_lang_deepl_for_answer, target_lang_detect='ko')
        print(f"최종 번역된 답변 ({target_lang_deepl_for_answer}): {final_output}")
        return final_output
    else:
        return output_ko # 원래 질문이 한국어였으면 번역 없이 반환

번역 통합 RAG 체인 생성 완료.


In [16]:
if lang_rag_chain_translate and translator:
    # 한국어 쿼리에 대한 성능 평가 (번역 불필요)
    query_ko_trans = "테슬라 창업자는 누구인가요?"
    print(f"\n--- 실행: 한국어 질문 '{query_ko_trans}' ---")
    output_ko_final = run_lang_rag_chain_with_translation(query_ko_trans)
    print(f"최종 결과 (KO): {output_ko_final}")

    # 영어 쿼리에 대한 성능 평가 (질문 KO로 번역 -> 답변 KO 생성 -> 답변 EN으로 번역)
    query_en_trans = "Who is the founder of Tesla?"
    print(f"\n--- 실행: 영어 질문 '{query_en_trans}' ---")
    output_en_final = run_lang_rag_chain_with_translation(query_en_trans)
    print(f"최종 결과 (EN): {output_en_final}")

    # 다른 언어 테스트 (예: 일본어, DeepL 무료 버전은 제한적일 수 있음)
    query_ja_trans = "テスラの創業者は誰ですか？"
    print(f"\n--- 실행: 일본어 질문 '{query_ja_trans}' ---")
    output_ja_final = run_lang_rag_chain_with_translation(query_ja_trans)
    print(f"최종 결과 (JA): {output_ja_final}")
else:
    print("\n번역 통합 RAG 체인 또는 DeepL 번역기가 준비되지 않아 실행을 건너뜁니다.")


--- 실행: 한국어 질문 '테슬라 창업자는 누구인가요?' ---
번역 불필요: 감지된 언어 'ko' (단순화: 'ko') == 목표 언어 'KO' (단순화: 'ko'). 원본 텍스트 반환.
원본 질문 (ko): 테슬라 창업자는 누구인가요?
RAG 답변 (KO): 마틴 에버하드와 마크 타페닝입니다.
최종 결과 (KO): 마틴 에버하드와 마크 타페닝입니다.

--- 실행: 영어 질문 'Who is the founder of Tesla?' ---
번역 필요: 감지된 언어 'en' -> 목표 언어 'KO'
원본 질문 (en): Who is the founder of Tesla?
번역된 질문 (KO): 테슬라의 창립자는 누구인가요?
RAG 답변 (KO): 마틴 에버하드와 마크 타페닝입니다.
번역 필요: 감지된 언어 'ko' -> 목표 언어 'EN-US'
최종 번역된 답변 (EN-US): Martin Everhard and Mark Tappening.
최종 결과 (EN): Martin Everhard and Mark Tappening.

--- 실행: 일본어 질문 'テスラの創業者は誰ですか？' ---
번역 필요: 감지된 언어 'ja' -> 목표 언어 'KO'
원본 질문 (ja): テスラの創業者は誰ですか？
번역된 질문 (KO): 테슬라의 창업자는 누구인가요?
RAG 답변 (KO): 마틴 에버하드와 마크 타페닝입니다.
번역 필요: 감지된 언어 'ko' -> 목표 언어 'JA'
최종 번역된 답변 (JA): マーティン・エバーハードとマーク・タフェニングです。
최종 결과 (JA): マーティン・エバーハードとマーク・タフェニングです。


### 1-3 언어 감지 및 벡터저장소 라우팅

- 이 전략은 각 언어별로 최적화된 문서 저장소와 임베딩 모델을 구축하고, 사용자 질문의 언어를 감지하여 해당 언어의 저장소로 요청을 라우팅(routing)하는 방식.

**전략:**
1.  **언어별 벡터 저장소 구축**: 한국어 문서는 한국어에 강한 임베딩 모델로, 영어 문서는 영어에 강한 (또는 좋은 다국어) 임베딩 모델로 각각 별도의 벡터 저장소를 만듬.
2.  **언어 감지**: 사용자 질문의 언어를 감지
3.  **라우팅**: 감지된 언어에 해당하는 벡터 저장소 및 관련 RAG 체인으로 질문을 전달
4.  **RAG 처리**: 선택된 RAG 체인에서 답변을 생성

**장점:**
- 각 언어에 최적화된 임베딩과 RAG 파이프라인을 사용할 수 있어, 해당 언어 내에서의 검색 및 답변 품질이 높을 수 있음.
- 번역 과정이 없어 지연 시간과 번역 비용이 발생하지 않음/ (단, 교차 언어 질의는 직접 지원하지 않음).

**단점:**
- **여러 벡터 저장소 관리**: 지원하는 언어 수만큼 벡터 저장소와 RAG 체인을 관리해야 하므로 복잡성이 증가.
- **교차 언어 검색 미지원**: 기본적으로는 질문 언어와 동일한 언어의 문서만 검색(예: 영어 질문으로는 영어 문서만 검색). 교차 언어 검색을 지원하려면 각 저장소에서 사용하는 임베딩 모델 자체가 교차 언어 성능이 뛰어나거나, 추가적인 번역 로직이 필요.
- **언어 감지 정확도 중요**: 언어 감지가 잘못되면 엉뚱한 저장소로 라우팅되어 성능이 저하

`(1) 언어별 벡터저장소 생성 `  

- 한국어 문서(`korean_docs_split`)와 영어 문서(`english_docs_split`)를 각각 다른 임베딩 모델(또는 동일한 고성능 다국어 모델)을 사용하여 별도의 ChromaDB 컬렉션에 저장

- 여기서는 예시로 한국어 문서는 HuggingFace의 `BAAI/bge-m3` (다국어 모델이지만 한국어 데이터에 대한 성능도 준수)를 사용하고, 영어 문서는 Ollama의 `bge-m3` (동일 모델을 Ollama 통해 사용)를 사용한다고 가정

In [17]:
# 한국어 문서, 영어 문서를 각각 다른 벡터 저장소에 저장
db_korean_route = None
db_english_route = None

# 한국어 문서용 벡터 저장소 (HuggingFace bge-m3 사용)
if korean_docs_split:
    print("한국어 문서용 벡터 저장소 생성 중 (HuggingFace bge-m3 사용)...")
    db_korean_route = Chroma.from_documents(
        documents=korean_docs_split, 
        embedding=embeddings_huggingface_bge_m3_cl, # 5-1에서 정의한 HuggingFace bge-m3 임베딩 재사용
        collection_name="db_korean_routed_v2",
        persist_directory="./chroma_db_routed", # 라우팅용 별도 디렉토리
    )
    print(f"한국어 벡터 저장소 문서 수: {db_korean_route._collection.count()}")
else:
    print("분할된 한국어 문서가 없어 한국어 벡터 저장소 생성을 건너뜁니다.")

# 영어 문서용 벡터 저장소 (Ollama bge-m3 사용)
if english_docs_split and ollama_ready_cl and embeddings_ollama_bge_m3_cl:
    print("\n영어 문서용 벡터 저장소 생성 중 (Ollama bge-m3 사용)...")
    db_english_route = Chroma.from_documents(
        documents=english_docs_split, 
        embedding=embeddings_ollama_bge_m3_cl, # 5-1에서 정의한 Ollama bge-m3 임베딩 재사용
        collection_name="db_english_routed_v2",
        persist_directory="./chroma_db_routed",
    )
    print(f"영어 벡터 저장소 문서 수: {db_english_route._collection.count()}")
elif not english_docs_split:
    print("\n분할된 영어 문서가 없어 영어 벡터 저장소 생성을 건너뜁니다.")
else:
    print("\nOllama가 준비되지 않아 영어 벡터 저장소(Ollama bge-m3) 생성을 건너뜁니다.")


한국어 문서용 벡터 저장소 생성 중 (HuggingFace bge-m3 사용)...
한국어 벡터 저장소 문서 수: 6

영어 문서용 벡터 저장소 생성 중 (Ollama bge-m3 사용)...
영어 벡터 저장소 문서 수: 4


`(2) RAG 체인 성능 평가 `  

언어 감지 결과를 바탕으로 적절한 RAG 체인(한국어용 또는 영어용)을 선택하여 실행하는 함수를 생성.

In [18]:
# 각 언어별 RAG 체인 생성 (create_rag_chain_cl 함수 재사용)
rag_chain_korean_routed = create_rag_chain_cl(db_korean_route)
rag_chain_english_routed = create_rag_chain_cl(db_english_route)

if rag_chain_korean_routed: print("라우팅용 한국어 RAG 체인 생성 완료")
else: print("라우팅용 한국어 RAG 체인 생성 실패 (벡터 저장소 없음)")

if rag_chain_english_routed: print("라우팅용 영어 RAG 체인 생성 완료")
else: print("라우팅용 영어 RAG 체인 생성 실패 (벡터 저장소 없음)")

def run_route_rag_chain(query: str):
    try:
        detected_query_lang = detect(query) # 질문 언어 감지
        print(f"감지된 질문 언어: {detected_query_lang}")
    except Exception as e:
        print(f"질문 언어 감지 실패: {e}. 기본 체인(한국어) 시도 또는 에러 반환.")
        # detected_query_lang = 'unknown' # 또는 기본 언어 설정
        return f"질문 언어 감지 실패: {e}"
    
    if detected_query_lang == 'ko' and rag_chain_korean_routed:
        print("한국어 RAG 체인으로 라우팅합니다.")
        return rag_chain_korean_routed.invoke(query)
    elif detected_query_lang == 'en' and rag_chain_english_routed:
        print("영어 RAG 체인으로 라우팅합니다.")
        return rag_chain_english_routed.invoke(query)
    else:
        # 지원하지 않는 언어 또는 해당 언어 체인이 없는 경우
        if detected_query_lang == 'ko' and not rag_chain_korean_routed:
            return "한국어 RAG 체인이 준비되지 않았습니다."
        if detected_query_lang == 'en' and not rag_chain_english_routed:
            return "영어 RAG 체인이 준비되지 않았습니다."
        return f"지원하지 않는 언어({detected_query_lang})이거나 해당 언어의 RAG 체인이 없습니다. 한국어 또는 영어로 질문해주세요."
    
# 한국어 쿼리에 대한 성능 평가
query_ko_route = "테슬라 창업자는 누구인가요?"
print(f"\n--- 실행: 라우팅 한국어 질문 '{query_ko_route}' ---")
output_ko_route = run_route_rag_chain(query_ko_route)
print(f"결과 (KO 라우팅): {output_ko_route}")

# 영어 쿼리에 대한 성능 평가
query_en_route = "Who is the founder of Tesla?"
print(f"\n--- 실행: 라우팅 영어 질문 '{query_en_route}' ---")
output_en_route = run_route_rag_chain(query_en_route)
print(f"결과 (EN 라우팅): {output_en_route}")

# 지원하지 않는 언어 테스트 (예: 일본어)
query_ja_route = "テスラの創業者は誰ですか？"
print(f"\n--- 실행: 라우팅 일본어 질문 '{query_ja_route}' ---")
output_ja_route = run_route_rag_chain(query_ja_route)
print(f"결과 (JA 라우팅): {output_ja_route}")

라우팅용 한국어 RAG 체인 생성 완료
라우팅용 영어 RAG 체인 생성 완료

--- 실행: 라우팅 한국어 질문 '테슬라 창업자는 누구인가요?' ---
감지된 질문 언어: ko
한국어 RAG 체인으로 라우팅합니다.
결과 (KO 라우팅): 마틴 에버하드와 마크 타페닝입니다.

--- 실행: 라우팅 영어 질문 'Who is the founder of Tesla?' ---
감지된 질문 언어: en
영어 RAG 체인으로 라우팅합니다.
결과 (EN 라우팅): Tesla was founded by Martin Eberhard and Marc Tarpenning.

--- 실행: 라우팅 일본어 질문 'テスラの創業者は誰ですか？' ---
감지된 질문 언어: ja
결과 (JA 라우팅): 지원하지 않는 언어(ja)이거나 해당 언어의 RAG 체인이 없습니다. 한국어 또는 영어로 질문해주세요.
